In [ ]:
import re
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup

countryDict = {
    'au': '.com.au',
    'at': '.at',
    'be': '.be',
    'ca': '.ca',
    'ch': '.ch',
    'de': '.de',
    'es': '.es',
    'fr': '.fr',
    'hk': '.com.hk',
    'ie': '.ie',
    'it': '.it',
    'my': '.com.my',
    'nl': '.nl',
    'nz': '.co.nz',
    'ph': '.ph',
    'pl': '.pl',
    'sg': '.com.sg',
    'uk': '.co.uk',
    'us': '.com',
}

conditionDict = {
    'all': '',
    'new': '&LH_ItemCondition=1000',
    'opened': '&LH_ItemCondition=1500',
    'refurbished': '&LH_ItemCondition=2500',
    'used': '&LH_ItemCondition=3000'
}

typeDict = {
    'all': '&LH_All=1',
    'auction': '&LH_Auction=1',
    'bin': '&LH_BIN=1',
    'offers': '&LH_BO=1'
}

def Items(url,Keyword):
    
    soup = __GetHTML(url)
    data = __ParseItems(soup,Keyword)
    
    return data

def __GetHTML(url):

    # Get the web page HTML
    request = urllib.request.urlopen(url)
    soup = BeautifulSoup(request.read(), 'html.parser')

    return soup

def __ParseItems(soup,Keyword):
    rawItems = soup.find_all('div', {'class': 's-item__info clearfix'})
    data = []

    for item in rawItems[1:]:
        
        #Get item data
        title = item.find(class_="s-item__title").find('span').get_text(strip=True)
        
        price = __ParseRawPrice(item.find('span', {'class': 's-item__price'}).get_text(strip=True))
        
        url = item.find('a')['href']

        itemData = {
            'Product': title,
            'price': price,
            'URL': url,
            'Keyword':Keyword,
        }
        
        data.append(itemData)
    
    # Remove item with prices too high or too low
    return data

def __ParsePrices(soup):
    
    # Get item prices
    rawPriceList = [price.get_text(strip=True) for price in soup.find_all(class_="s-item__price")]
    priceList = [price for price in map(lambda rawPrice:__ParseRawPrice(rawPrice), rawPriceList) if price != None]
    
    # Get shipping prices
    rawShippingList = [item.get_text(strip=True) for item in soup.find_all(class_="s-item__shipping s-item__logisticsCost")]
    shippingList = map(lambda rawPrice:__ParseRawPrice(rawPrice), rawShippingList)
    shippingList = [0 if price == None else price for price in shippingList]

    # Remove prices too high or too low
    priceList = __StDevParse(priceList)
    shippingList = __StDevParse(shippingList)

    data = {
        'price-list': priceList,
        'shipping-list': shippingList
    }
    return data

def __ParseRawPrice(string):
    parsedPrice = re.search('(\d+(.\d+)?)', string.replace(',', '.'))
    if (parsedPrice):
        return float(parsedPrice.group())
    else:
        return None

def __Average(numberList):

    if len(list(numberList)) == 0: return 0
    return sum(numberList) / len(list(numberList))

def __StDev(numberList):
    
    if len(list(numberList)) <= 1: return 0
    
    nominator = sum(map(lambda x: (x - sum(numberList) / len(numberList)) ** 2, numberList))
    stdev = (nominator / ( len(numberList) - 1)) ** 0.5

    return stdev

def __StDevParse(numberList):
    
    avg = __Average(numberList)
    stdev = __StDev(numberList)
    
    # Remove prices too high or too low; Accept Between -1 StDev to +1 StDev
    numberList = [nmbr for nmbr in numberList if (avg + stdev >= nmbr >= avg - stdev)]

    return numberList


def __GetaAllClasses(soup):
    classes = set()
    for element in soup.find_all(True, class_=True):
        classes.update(element.get('class'))

    return classes

In [52]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup

# Specify the path to your Excel file (without the ~$ prefix)
excel_file_path = 'Files/Ebaycom-Input.xlsx'
output_file_path = 'Files/OutputFile.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)

# Assuming the links are in a column named 'URL' and keywords in 'Keyword'
links = df['URL'].tolist()
keywords = df['Keyword'].tolist()

data = []  # List to hold all the processed data
for link,keyword in zip(links,keywords):
    item_data = Items(link,keyword)
    
    data.append(item_data)

# Convert the collected data into a DataFrame
result_df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
result_df.to_excel(output_file_path, index=False)  # index=False to not write row indices

print("Data has been saved to", output_file_path)


Data has been saved to Files/OutputFile.xlsx
